# Introduction

put stuff here

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
import requests
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"}

In [2]:
# List of state abbreviations
states = ['AL', 'AK', 'AZ' ,'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN',
 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV',
 'NH', 'NJ', 'NM', 'NY', 'DC', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD',
 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']


In [3]:
list = []
x = 0
#df = pd.json_normalize(res.json())
for state in states:
    res = requests.get(f'https://chronicdata.cdc.gov/resource/eav7-hnsx.json?measureid=MHLTH&$limit=4000&stateabbr={state}')
    list.append(res.json())
    res = requests.get(f'https://chronicdata.cdc.gov/resource/eav7-hnsx.json?measureid=DEPRESSION&$limit=4000&stateabbr={state}')
    list.append(res.json())
    
    

In [4]:
df = pd.json_normalize(list[0])
df.set_index('locationid')
for i in range(1,len(list)):
    curr = pd.json_normalize(list[i])
    df = df.append(curr)


In [5]:
# Cleaning up DF a little bit

df = df.rename(columns={'statedesc': 'State', 'locationname': 'City', 'measure': 'Symptom', 'data_value': 'Percent'})
del df['datasource']
del df['measureid']
del df['geolocation.type']
del df['geolocation.coordinates']
del df['data_value_footnote_symbol']
del df['data_value_footnote']
del df['low_confidence_limit']
del df['high_confidence_limit']
del df['categoryid']
del df['datavaluetypeid']

In [6]:
# Import Income Data Set
income_df = pd.read_csv("kaggle_income.csv", encoding='latin-1')
income_df['loc_fullname'] = income_df['City'] + ', ' + income_df['State_Name']

# delete unnecessary columns
del income_df['id']
del income_df['Type']
del income_df['County']
del income_df['Place']
del income_df['Primary']
del income_df['Lat']
del income_df['Lon']
del income_df['sum_w']
del income_df['Stdev']
del income_df['Area_Code']
del income_df['ALand']
del income_df['AWater']


# rename columns
income_df = income_df.rename(columns = {'Mean':'Mean Income','Median':'Median Income'})

# combine duplicate cities
grouped = income_df.groupby(['loc_fullname'], as_index=False)
income_df = grouped.agg('mean')

In [7]:
# import us cities dataset
cities_df = pd.read_csv('uscities.csv')
del cities_df['city_ascii']
del cities_df['lat']
del cities_df['lng']
del cities_df['source']
del cities_df['military']
del cities_df['incorporated']
del cities_df['timezone']
del cities_df['ranking']
del cities_df['id']

cities_df = cities_df.rename(columns={'population': 'Total Population'})
cities_df['loc_fullname'] = cities_df['city'] + ', ' + cities_df['state_name']
cities_df['county_name'] = cities_df['county_name'] + ' County'

cities_df.head(10)

,city,state_id,state_name,county_fips,county_name,Total Population,density,zips,loc_fullname
0,New York,NY,New York,36061,New York County,18713220,10715,11229 11226 11225 11224 11222 11221 11220 1138...,"New York, New York"
1,Los Angeles,CA,California,6037,Los Angeles County,12750807,3276,90291 90293 90292 91316 91311 90037 90031 9000...,"Los Angeles, California"
2,Chicago,IL,Illinois,17031,Cook County,8604203,4574,60018 60649 60641 60640 60643 60642 60645 6064...,"Chicago, Illinois"
3,Miami,FL,Florida,12086,Miami-Dade County,6445545,5019,33129 33125 33126 33127 33128 33149 33144 3314...,"Miami, Florida"
4,Dallas,TX,Texas,48113,Dallas County,5743938,1526,75287 75098 75233 75254 75251 75252 75253 7503...,"Dallas, Texas"
5,Philadelphia,PA,Pennsylvania,42101,Philadelphia County,5649300,4554,19154 19151 19150 19153 19152 19102 19103 1910...,"Philadelphia, Pennsylvania"
6,Houston,TX,Texas,48201,Harris County,5464251,1399,77069 77068 77061 77060 77063 77062 77065 7706...,"Houston, Texas"
7,Atlanta,GA,Georgia,13121,Fulton County,5449398,1441,30334 30331 30332 30309 30308 30305 30307 3030...,"Atlanta, Georgia"
8,Washington,DC,District of Columbia,11001,District of Columbia County,5379184,4457,20010 20011 20012 20015 20228 20520 20307 2041...,"Washington, District of Columbia"
9,Boston,MA,Massachusetts,25025,Suffolk County,4688346,5532,02120 02121 02122 02124 02125 02126 02127 0212...,"Boston, Massachusetts"


In [8]:
# lets also do temperature by county, import from this dataset

temperature = pd.read_csv('temp_by_county.csv')
temperature

# this data is formatted pretty weird
# lets add a state column so we can more reasily merge it back onto our main df

temperature['State'] = temperature['Location ID'].apply(lambda x: x[:2])
temperature

,Location ID,Location,Value,Rank,Anomaly (1901-2000 base period),1901-2000 Mean,State
0,AL-001,Autauga County,53.0,48,-1.3,54.3,AL
1,AL-003,Baldwin County,57.6,61,-0.7,58.3,AL
2,AL-005,Barbour County,54.6,47,-1.2,55.8,AL
3,AL-007,Bibb County,51.2,44,-1.6,52.8,AL
4,AL-009,Blount County,50.3,61,-0.3,50.6,AL
...,...,...,...,...,...,...,...
3131,AK-240,Southeast Fairbanks Census Area,-0.8,21,-5.0,4.2,AK
3132,AK-261,Valdez-Cordova Census Area,7.1,14,-6.2,13.3,AK
3133,AK-275,Wrangell City and Borough,32.6,52,0.8,31.8,AK
3134,AK-282,Yakutat City and Borough,20.9,24,-3.1,24.0,AK


In [9]:
# Create a column that holds: {City}, {State}
# We will merge using this as the key. Then we will delete this (because we already have columns for state and cities)

df['loc_fullname'] = df['City'] + ', ' + df['State']
df = df.merge(income_df[['loc_fullname', 'Mean Income', 'Median Income', 'Zip_Code']], left_on='loc_fullname', right_on='loc_fullname')
df = df.merge(cities_df[['loc_fullname', 'Total Population', 'county_name']])
df = df.merge(temperature, left_on=['stateabbr', 'county_name'], right_on=['State', 'Location'])
del df['loc_fullname']

In [10]:
df['Mental Health Age-Adjusted Percent'] = (df.loc[((df['short_question_text'] == 'Mental Health') & (df['data_value_type'] == 'Age-adjusted prevalence'))]['Percent']).astype('double')
df['Mental Health Crude Percent'] = (df.loc[((df['short_question_text'] == 'Mental Health') & (df['data_value_type'] == 'Crude prevalence'))]['Percent']).astype('double')
df['Depression Age-Adjusted Percent'] = (df.loc[((df['short_question_text'] == 'Depression') & (df['data_value_type'] == 'Age-adjusted prevalence'))]['Percent']).astype('double')
df['Depression Crude Percent'] = (df.loc[((df['short_question_text'] == 'Depression') & (df['data_value_type'] == 'Crude prevalence'))]['Percent']).astype('double')


In [11]:
df.head(5)

,year,stateabbr,State_x,City,category,Symptom,data_value_unit,data_value_type,Percent,totalpopulation,...,Location,Value,Rank,Anomaly (1901-2000 base period),1901-2000 Mean,State_y,Mental Health Age-Adjusted Percent,Mental Health Crude Percent,Depression Age-Adjusted Percent,Depression Crude Percent
0,2019,AL,Alabama,Childersburg,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,21.7,5175,...,Talladega County,50.8,50,-1.0,51.8,AL,21.7,NaN,NaN,NaN
1,2019,AL,Alabama,Childersburg,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,21.2,5175,...,Talladega County,50.8,50,-1.0,51.8,AL,NaN,21.2,NaN,NaN
2,2019,AL,Alabama,Childersburg,Health Outcomes,Depression among adults aged >=18 years,%,Age-adjusted prevalence,27.5,5175,...,Talladega County,50.8,50,-1.0,51.8,AL,NaN,NaN,27.5,NaN
3,2019,AL,Alabama,Childersburg,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,27.3,5175,...,Talladega County,50.8,50,-1.0,51.8,AL,NaN,NaN,NaN,27.3
4,2019,AL,Alabama,Sylacauga,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,19.7,12831,...,Talladega County,50.8,50,-1.0,51.8,AL,NaN,19.7,NaN,NaN


In [12]:
df.dtypes

year                                   object
stateabbr                              object
State_x                                object
City                                   object
category                               object
Symptom                                object
data_value_unit                        object
data_value_type                        object
Percent                                object
totalpopulation                        object
locationid                             object
short_question_text                    object
Mean Income                           float64
Median Income                         float64
Zip_Code                              float64
Total Population                        int64
county_name                            object
Location ID                            object
Location                               object
Value                                 float64
Rank                                    int64
Anomaly (1901-2000 base period)   

In [13]:
del df['Symptom']
del df['data_value_unit']
del df['data_value_type']
del df['Percent']
del df['short_question_text']
del df['Location ID']
del df['Location']
del df['Value']
del df['Rank']
del df['State_y']
del df['Anomaly (1901-2000 base period)']
del df['category']

df = df.rename(columns={'State_x': 'State', '1901-2000 Mean': 'Yearly Avg Temp'})

In [14]:
# Aggregate rows to have one instance of each and information on mental health and depression rates
grouped = df.groupby(by=['year','State', 'stateabbr','City','totalpopulation','locationid','Mean Income','Median Income','Zip_Code','Total Population', 'county_name', 'Yearly Avg Temp'], as_index=False, dropna=False)
df = grouped.agg('sum')

In [15]:
df

,year,State,stateabbr,City,totalpopulation,locationid,Mean Income,Median Income,Zip_Code,Total Population,county_name,Yearly Avg Temp,Mental Health Age-Adjusted Percent,Mental Health Crude Percent,Depression Age-Adjusted Percent,Depression Crude Percent
0,2019,Alabama,AL,Abbeville,2688,0100124,40518.000000,25216.000000,36310.0,2560,Henry County,56.8,20.2,18.3,24.8,23.6
1,2019,Alabama,AL,Adamsville,4522,0100460,53383.000000,47460.000000,35005.0,4281,Jefferson County,51.7,18.8,18.1,23.6,23.3
2,2019,Alabama,AL,Alabaster,31367,0100820,81118.333333,149711.333333,35007.0,33487,Shelby County,52.1,15.4,15.6,23.3,23.7
3,2019,Alabama,AL,Albertville,21282,0100988,47199.500000,160358.500000,35950.5,37775,Marshall County,50.5,19.4,19.5,28.2,28.3
4,2019,Alabama,AL,Alexander City,14972,0101132,40286.000000,31306.000000,35010.0,14317,Tallapoosa County,52.8,19.5,18.6,26.1,25.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8560,2019,Wyoming,WY,Upton,1100,5679125,69215.000000,300000.000000,82730.0,1056,Weston County,31.1,14.4,13.8,18.8,18.2
8561,2019,Wyoming,WY,Wamsutter,451,5681300,0.000000,0.000000,82336.0,467,Sweetwater County,28.5,12.4,12.8,17.4,18.3
8562,2019,Wyoming,WY,Wheatland,3627,5683040,87954.000000,164106.000000,82201.0,3462,Platte County,33.7,14.0,12.8,19.6,18.6
8563,2019,Wyoming,WY,Worland,5487,5684925,61427.000000,300000.000000,82401.0,5024,Washakie County,30.8,14.1,13.3,18.7,18.1


In [16]:
df.set_index('locationid',inplace = True)

In [17]:
df.loc[df['Mean Income'].astype('int') < 1000]
# As per our data source, 0 income means that there are no records of income for these cities. 
# We can either make these values NAN or completely get rid of these rows. 
# We decided to just delete these rows as none of them are extremely 'important' cities. 

,year,State,stateabbr,City,totalpopulation,Mean Income,Median Income,Zip_Code,Total Population,county_name,Yearly Avg Temp,Mental Health Age-Adjusted Percent,Mental Health Crude Percent,Depression Age-Adjusted Percent,Depression Crude Percent
locationid,,,,,,,,,,,,,,,
0179008,2019,Alabama,AL,Vincent,2082,0.0,0.0,35178.0,2213,Shelby County,52.1,20.6,19.8,26.6,26.2
0418160,2019,Arizona,AZ,Dateland,416,0.0,0.0,85333.0,434,Yuma County,60.7,13.8,13.3,17.3,17.2
0603302,2019,California,CA,Avenal,15505,0.0,0.0,93204.0,13496,Kings County,53.2,16.9,18.2,16.6,17.2
0823300,2019,Colorado,CO,Edwards,10266,0.0,0.0,81632.0,9350,Eagle County,27.8,12.9,13.5,18.1,18.9
1048330,2019,Delaware,DE,Milton,2576,0.0,0.0,19968.0,3012,Sussex County,46.5,13.9,13.4,18.8,18.6
1211225,2019,Florida,FL,Cedar Key,706,0.0,0.0,32625.0,720,Levy County,62.1,19.3,15.8,23.2,20.8
1251075,2019,Florida,FL,Ocoee,35827,0.0,0.0,34761.0,48263,Orange County,66.1,16.0,16.3,20.1,20.4
1723737,2019,Illinois,IL,El Paso,2810,0.0,0.0,61738.0,2730,Woodford County,38.7,15.3,14.7,23.1,22.5
1768640,2019,Illinois,IL,Seneca,2371,0.0,0.0,61360.0,2251,LaSalle County,38.2,15.5,15.3,22.6,22.6


In [18]:
df = df.drop(df[df['Mean Income'].astype('int') < 1000].index)
df

,year,State,stateabbr,City,totalpopulation,Mean Income,Median Income,Zip_Code,Total Population,county_name,Yearly Avg Temp,Mental Health Age-Adjusted Percent,Mental Health Crude Percent,Depression Age-Adjusted Percent,Depression Crude Percent
locationid,,,,,,,,,,,,,,,
0100124,2019,Alabama,AL,Abbeville,2688,40518.000000,25216.000000,36310.0,2560,Henry County,56.8,20.2,18.3,24.8,23.6
0100460,2019,Alabama,AL,Adamsville,4522,53383.000000,47460.000000,35005.0,4281,Jefferson County,51.7,18.8,18.1,23.6,23.3
0100820,2019,Alabama,AL,Alabaster,31367,81118.333333,149711.333333,35007.0,33487,Shelby County,52.1,15.4,15.6,23.3,23.7
0100988,2019,Alabama,AL,Albertville,21282,47199.500000,160358.500000,35950.5,37775,Marshall County,50.5,19.4,19.5,28.2,28.3
0101132,2019,Alabama,AL,Alexander City,14972,40286.000000,31306.000000,35010.0,14317,Tallapoosa County,52.8,19.5,18.6,26.1,25.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5677530,2019,Wyoming,WY,Torrington,6501,47383.000000,171237.500000,82240.0,7289,Goshen County,34.0,13.8,13.2,18.9,18.3
5679125,2019,Wyoming,WY,Upton,1100,69215.000000,300000.000000,82730.0,1056,Weston County,31.1,14.4,13.8,18.8,18.2
5683040,2019,Wyoming,WY,Wheatland,3627,87954.000000,164106.000000,82201.0,3462,Platte County,33.7,14.0,12.8,19.6,18.6


In [19]:
# That was all collection and processing. Now exploratory analysis and Visualization

# Some things we could possibly explore in analysis
    # most depressed cities
    # avg depression/mental health rate by city
    # state with worst depression, state with worst mental health
    # income vs depression rates
    # population density vs depression/mental health rates
    # temp vs depression/mental health rates
    # for every city, mental health rates vs depression rates, similar?
    # maybe try using folium, geopandas, basemap. Couldn't get any to work atm
        # talk about regions that seem to have higher rates of mental illness/depression
        # if we dont do this we could also keep longitude/latitude data, plot a distribution